In [ ]:
import pandas as pd
import numpy as np

invoice_c = pd.read_excel('Courier Company - Invoice.xlsx')
invoice_c

rates_c = pd.read_excel('Courier Company - Rates.xlsx')
rates_c

order_x = pd.read_excel('Company X - Order Report.xlsx')
order_x

pin_x = pd.read_excel('Company X - Pincode Zones.xlsx')
pin_x

sku_x = pd.read_excel('Company X - SKU Master.xlsx')
sku_x

new = invoice_c[['Order ID','AWB Code','Warehouse Pincode','Customer Pincode','Type of Shipment']]
new

df_add = pd.merge(order_x,sku_x, on='SKU', how='inner')
df_add

df_add['total'] = df_add['Weight (g)']*df_add['Order Qty'] / 1000
df_add

##### There are 2 options right now as I can do
(1) As there are duplicates in df_add ExternOrderNo I can group by find sum total of each unique customer

(2) Or I can left them as this and add this after I do weight slab

As when I recive any order if I have more than 2 order I get different box for each other so I am going to group them after I weight slab them

# df_add.to_excel('group.xlsx')

gp_df = df_add.copy()

gp_df = gp_df.groupby('ExternOrderNo').sum('total')

gp_df

gp_df.drop(['Order Qty','Weight (g)'], axis=1, inplace=True)
gp_df

gp_df = pd.merge(gp_df,new,left_on='ExternOrderNo',right_on = 'Order ID', how='inner')
gp_df

invoice_c[invoice_c['Order ID']==2001823564]

k_df = gp_df.copy()

pin_x

pin_x[pin_x.duplicated('Customer Pincode')]

pin_x[pin_x['Customer Pincode']==302020]

pin_x.drop_duplicates('Customer Pincode', inplace=True)

pin_x[pin_x['Customer Pincode']==302020]

k_df = pd.merge(k_df,pin_x,on='Customer Pincode', how='inner')
k_df

k_new = k_df.copy()

k_new = k_new.drop(['Warehouse Pincode_x','Customer Pincode', 'Warehouse Pincode_y'], axis=1)
k_new

k_new['Zone'] = k_new['Zone'].str.upper()

k_new

k_new = pd.merge(k_new,rates_c,on='Zone')
k_new

def calculate_x_charge(k_new):
    
    x = k_new['total']/k_new['Weight Slabs']
    x=int(x)
    if(k_new['total']%k_new['Weight Slabs']==0):
        x = x - 1
    y=0
    if(k_new['Type of Shipment']== 'Forward and RTO charges'):
        y=1
    Charge = k_new['Forward Fixed Charge'] + x*k_new['Forward Additional Weight Slab Charge']  + y*k_new['RTO Fixed Charge'] + y*x*k_new['RTO Additional Weight Slab Charge']
    return Charge
                                                                                              

k_new['Charge_X'] = k_new.apply(calculate_x_charge, axis=1)
k_new

k1_n = k_new.copy()

def weight_slab(k1_n):
    x = k1_n['total'] / k1_n['Weight Slabs']
    x = int(x)
    if(k1_n['total']% k1_n['Weight Slabs']==0):
        x = x-1
    weight = k1_n['Weight Slabs'] + x*k1_n['Weight Slabs']
    return weight

k1_n['Weight slab per X'] = k1_n.apply(weight_slab,axis=1)
k1_n



rates_c

invoice_c

cour_c = invoice_c[['Order ID', 'AWB Code', 'Charged Weight', 'Zone']]

cour_c['Zone'] = cour_c['Zone'].str.upper()

cour_c = pd.merge(cour_c,rates_c,on='Zone')
cour_c

def weight_slab(cour_c):
    x = cour_c['Charged Weight'] / cour_c['Weight Slabs']
    x = int(x)
    if(cour_c['Charged Weight']% cour_c['Weight Slabs']==0):
        x = x-1
    weight = cour_c['Weight Slabs'] + x*cour_c['Weight Slabs']
    return weight

# Made by Kartik Kothari

cour_c['Weight Slab Charge by Courier'] = cour_c.apply(weight_slab,axis=1)
cour_c

cour_c['Types of Shipment'] = invoice_c['Type of Shipment']
cour_c

def calculate_x_charge(cour_c):
    
    x = cour_c['Charged Weight'] / cour_c['Weight Slabs']
    x=int(x)
    if(cour_c['Charged Weight']%cour_c['Weight Slabs']==0):
        x = x - 1
    y=0
    if(cour_c['Types of Shipment']== 'Forward and RTO charges'):
        y=1
    Charge = cour_c['Forward Fixed Charge'] + x*cour_c['Forward Additional Weight Slab Charge']  + y*cour_c['RTO Fixed Charge'] + y*x*cour_c['RTO Additional Weight Slab Charge']     #KK
    return Charge
                                                                                              

cour_c['Delivery Zone Charged by Courier Company'] = cour_c.apply(calculate_x_charge,axis=1)
cour_c

# cour_c.to_excel('Charge_Courier.xlsx')

k1_n

k2_n = k1_n.copy()

k2_n['total'] = k2_n['total'].round(2)
k2_n

brand_new = pd.DataFrame()

brand_new[['Order ID','AWB Code','Total Weight as per X (KG)','Weight Slab as per X (KG)','Delivery Zone as per X','Expected Charge as per X (Rs.)']] = k2_n[['Order ID','AWB Code','total','Weight slab per X','Zone','Charge_X']]

brand_new

cour_c

for_add_cour = pd.DataFrame()

for_add_cour[['Order ID', 'Total Weight as per Courier Company (KG)', 'Weight Slab Charged by Courier Company (KG)', 'Delivery Zone as per Courier Company', 'Zone Charged by Courier Company']] = cour_c[['Order ID', 'Charged Weight', 'Weight Slab Charge by Courier', 'Zone', 'Delivery Zone Charged by Courier Company']]

for_add_cour

br_new = pd.DataFrame()

br_new = pd.merge(brand_new,for_add_cour, on='Order ID', how='inner')
br_new

a1 = invoice_c[['Order ID','Billing Amount (Rs.)']]
a1

bm_new = pd.merge(br_new,a1,on='Order ID', how='inner')
bm_new

bm_new['Charged Billed by Courier Company (Rs.)'] = bm_new['Billing Amount (Rs.)']
bm_new

bm_new.drop(['Billing Amount (Rs.)'], axis=1, inplace=True)

bm_new

# bm_new.to_excel('Final_before_arra.xlsx')

bm_new['Difference Between Expected Charges and Billed Charges (Rs.)'] = bm_new['Expected Charge as per X (Rs.)'] - bm_new['Charged Billed by Courier Company (Rs.)']
bm_new

bm_new1 = bm_new.copy()

bm_new1.drop(['Zone Charged by Courier Company'],axis=1, inplace=True)

new_col_name = {'Delivery Zone as per Courier Company':'Delivery Zone charged by Courier Company'}
bm_new1.rename(columns = new_col_name, inplace=True)
bm_new1

bm_new1.columns

new_col_order = ['Order ID', 'AWB Code', 'Total Weight as per X (KG)','Weight Slab as per X (KG)',
                'Total Weight as per Courier Company (KG)',
       'Weight Slab Charged by Courier Company (KG)',
        'Delivery Zone as per X',
        'Delivery Zone charged by Courier Company',
       'Expected Charge as per X (Rs.)',
       'Charged Billed by Courier Company (Rs.)',
       'Difference Between Expected Charges and Billed Charges (Rs.)']

bm_new1 = bm_new1[new_col_order]
bm_new1

# bm_new1.to_excel('Result Order level calculation.xlsx',index=False)

count=[]
count.append((bm_new1['Difference Between Expected Charges and Billed Charges (Rs.)']==0).sum())
count.append((bm_new1['Difference Between Expected Charges and Billed Charges (Rs.)']<0).sum())
count.append((bm_new1['Difference Between Expected Charges and Billed Charges (Rs.)']>0).sum())
count

bm_new1[bm_new1['Difference Between Expected Charges and Billed Charges (Rs.)']==0]

bm_new1[bm_new1['Difference Between Expected Charges and Billed Charges (Rs.)']==0]['Expected Charge as per X (Rs.)'].sum().round(1)

amount=[]
amount.append(bm_new1[bm_new1['Difference Between Expected Charges and Billed Charges (Rs.)']==0]['Expected Charge as per X (Rs.)'].sum().round(1))
amount.append(abs(bm_new1[bm_new1['Difference Between Expected Charges and Billed Charges (Rs.)']<0]['Difference Between Expected Charges and Billed Charges (Rs.)'].sum().round(1)))
amount.append(bm_new1[bm_new1['Difference Between Expected Charges and Billed Charges (Rs.)']>0]['Difference Between Expected Charges and Billed Charges (Rs.)'].sum().round(1))
amount



data = {
    'Category':['Total orders where X has been correctly charged','Total Orders where X has been overcharged','Total Orders where X has been undercharged'],
    'Count':count,
    'Amount (Rs.)':amount,
}

summary

summary= pd.DataFrame(data,index=None)

summary

# summary.to_excel('summary_table.xlsx',index=False)







brand_new

invoice_c

bm_new1

plot_final = bm_new1.copy() 

import matplotlib.pyplot as plt

plt.scatter(plot_final['Order ID'],plot_final['Expected Charge as per X (Rs.)'],color='blue',marker='o',label='Expected Charge')
plt.scatter(plot_final['Order ID'],plot_final['Charged Billed by Courier Company (Rs.)'],color='salmon',marker='^',label='Courier Charge')
plt.xlabel('Order ID')
plt.ylabel('Charge (Rs.)')
plt.title('Scatter Plot between Order ID and Charge')
plt.legend()
plt.show()

##### It shows for same Order ID charge estimate by X and Courier Charge

##### Courier Charge is much more we can also evalute based on Zone, Weight Slab

plot_final['Order ID'].min()

plot_final['Order ID'].max()

plt.bar(plot_final['Delivery Zone as per X'],plot_final['Expected Charge as per X (Rs.)'])
plt.show()

plt.bar(plot_final['Delivery Zone charged by Courier Company'],plot_final['Charged Billed by Courier Company (Rs.)'],color='gray')
plt.show()

sum_by_zone = plot_final.groupby('Delivery Zone as per X').sum()
plt.figure(figsize=(10, 6))

plt.bar(sum_by_zone.index, sum_by_zone['Expected Charge as per X (Rs.)'], label='Expected Charge', alpha=0.7)
plt.bar(sum_by_zone.index, sum_by_zone['Charged Billed by Courier Company (Rs.)'], label='Charged by Courier', alpha=0.7, bottom=sum_by_zone['Expected Charge as per X (Rs.)'])

plt.xlabel('Delivery Zone')
plt.ylabel('Total Charges (Rs.)')
plt.title('Total Charges by Delivery Zone (B, C, D, E)')
plt.legend()

plt.show()





